In [1]:
from sklearn import linear_model
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import KFold,RepeatedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [2]:
#%%将lgb和xgb和ctb的结果进行stacking

#读取stacking数据
out_dir = "result/"
oof_lgb1=np.load(out_dir+"lgb_result/2020-05-10-22-12-44/oof_lgb.npy")
oof_lgb2=np.load(out_dir+"lgb2_result/2020-05-10-22-18-49/oof_lgb.npy")
oof_lgb3=np.load(out_dir+"lgb3_result/2020-05-10-22-19-31/oof_lgb.npy")
oof_lgb4=np.load(out_dir+"lgb4_result/2020-05-10-22-23-02/oof_lgb.npy")
oof_xgb=np.load(out_dir+"xgb_result/2020-05-10-23-03-05/oof_xgb.npy")
oof_xgb2=np.load(out_dir+"xgb2_result/2020-05-10-23-26-58/oof_xgb.npy")
oof_xgb3=np.load(out_dir+"xgb3_result/2020-05-12-09-57-49/oof_xgb.npy")
oof_xgb4=np.load(out_dir+"xgb4_result/2020-05-12-10-22-35/oof_xgb.npy")
oof_gbdt=np.load(out_dir+"gbdt_result/2020-05-10-18-54-49/oof_gbdt.npy")
oof_gbdt2=np.load(out_dir+"gbdt2_result/2020-05-12-13-15-56/oof_gbdt.npy")


prediction_lgb1=np.load(out_dir +"lgb_result/2020-05-10-22-12-44/prediction_lgb.npy")
prediction_lgb2=np.load(out_dir +"lgb2_result/2020-05-10-22-18-49/prediction_lgb.npy")
prediction_lgb3=np.load(out_dir +"lgb3_result/2020-05-10-22-19-31/prediction_lgb.npy")
prediction_lgb4=np.load(out_dir +"lgb4_result/2020-05-10-22-23-02/prediction_lgb.npy")
prediction_xgb=np.load(out_dir +"xgb_result/2020-05-10-23-03-05/prediction_xgb.npy")
prediction_xgb2=np.load(out_dir+"xgb2_result/2020-05-10-23-26-58/prediction_xgb.npy")
prediction_xgb3=np.load(out_dir+"xgb3_result/2020-05-12-09-57-49/prediction_xgb.npy")
prediction_xgb4=np.load(out_dir+"xgb4_result/2020-05-12-10-22-35/prediction_xgb.npy")
prediction_gbdt=np.load(out_dir+"gbdt_result/2020-05-10-18-54-49/prediction_gbdt.npy")
prediction_gbdt2=np.load(out_dir+"gbdt2_result/2020-05-12-13-15-56/prediction_gbdt.npy")



#读取训练集和测试集
train=pd.read_csv('data/train1.csv')
test=pd.read_csv('data/test1.csv')


In [13]:
features=list(train.columns)
features.remove('flag')
X_train, y_train=train[features],train['flag']
X_test = test[features]

In [14]:
prediction=pd.DataFrame(test['id'])

In [15]:
#%%构建stack模型
start = time.clock()
train_stack = np.vstack([oof_lgb1,oof_lgb2, oof_lgb3, oof_lgb4, oof_xgb, oof_xgb2, oof_xgb3, oof_xgb4, oof_gbdt, oof_gbdt2]).transpose()
test_stack = np.vstack([prediction_lgb1,prediction_lgb2, prediction_lgb3, prediction_lgb4, prediction_xgb, prediction_xgb2, prediction_xgb3, prediction_xgb4, prediction_gbdt, prediction_gbdt2]).transpose()


folds_stack = RepeatedKFold(n_splits=5, n_repeats=2, random_state=2019)
oof_stack = np.zeros(train_stack.shape[0])
prediction_stack = np.zeros(test_stack.shape[0])

result=np.zeros((X_train.shape[0], ))
for fold_, (trn_idx, val_idx) in enumerate(folds_stack.split(train_stack,y_train)):
    print("fold {}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], y_train[trn_idx]
    val_data, val_y = train_stack[val_idx], y_train[val_idx]
    
    clf_3 = linear_model.BayesianRidge(alpha_1=10, lambda_1=10)
    #clf_3 =linear_model.Ridge()
    clf_3.fit(trn_data, trn_y)
    
    oof_stack[val_idx] = clf_3.predict(val_data)
    threshold = 0.5
    for i in val_idx:
        result[i] = 1 if oof_stack[i] > threshold else 0
    prediction_stack += clf_3.predict(test_stack) / 10

print("CV score(ACC): {:<8.5f}".format(accuracy_score(y_train, result)))
print("CV score(AUC): {:<8.5f}".format(roc_auc_score(y_train, oof_stack)))
print ('CV score(F1): %.4f' % f1_score(y_train, result,average='weighted'))

fold 0
fold 1
fold 2
fold 3
fold 4
fold 5
fold 6
fold 7
fold 8
fold 9
CV score(ACC): 0.87496 
CV score(AUC): 0.92687 
CV score(F1): 0.8743


In [17]:
prediction['pred']=prediction_stack

In [8]:
prediction

,id,pred
0,U1E63A3,0.316861
1,U3EC77F,0.203708
2,U27DC0C,0.073599
3,U3FAAAC,0.085639
4,U04EA0B,0.074116
...,...,...
3995,U0FBFE2,0.030287
3996,U506AD2,0.089982
3997,U287B33,0.111454
3998,U15134F,0.123669


In [18]:
prediction.to_csv('result/predict_stack.txt', sep = '\t', index = 0, header = 0)

4000